In [3]:
import pandas as pd
import numpy as np
import torch
from torch import nn as nn

In [10]:
data = pd.read_csv('alphanet_codetest_daily.csv')
data['timestamp'] = pd.to_datetime(data.timestamp.astype('str'))
data['unix'] = data['timestamp'].astype('int64')//1e9
one_data = data[data['ticker'] == 1]
data

,timestamp,ticker,pre_close,open,close,high,low,vwap,chg,pct_chg,volume,amount,turnover,free_turnover,fq_factor,high_limit,low_limit,trading_status,unix
0,2019-09-23,1,15.34,15.34,15.38,15.47,15.18,15.3122,0.04,0.2608,1403282.00,2148740.448,0.7428,1.5038,109.169427,16.87,13.81,-1.0,1569196800
1,2019-09-23,2,26.80,26.49,26.15,26.49,26.00,26.1471,-0.65,-2.4254,603530.46,1578059.535,0.6212,1.0882,148.411917,29.48,24.12,-1.0,1569196800
2,2019-09-23,4,21.03,20.36,20.04,20.65,19.61,20.0171,-0.99,-4.7076,29893.32,59837.839,3.6188,5.9394,4.063862,23.13,18.93,-1.0,1569196800
3,2019-09-23,5,3.37,3.37,3.30,3.37,3.27,3.2947,-0.07,-2.0772,111437.30,36714.744,1.0533,1.3495,9.267603,3.71,3.03,-1.0,1569196800
4,2019-09-23,6,5.32,5.34,5.38,5.40,5.21,5.2993,0.06,1.1278,136997.47,72599.425,1.0161,1.5887,35.425960,5.85,4.79,-1.0,1569196800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1958382,2021-09-23,688798,234.00,234.10,240.33,241.79,230.51,237.5860,6.33,2.7051,5033.35,119585.342,1.5740,1.5740,1.000000,280.80,187.20,-1.0,1632355200
1958383,2021-09-23,688799,40.38,40.23,40.00,40.99,39.88,40.2658,-0.38,-0.9411,6743.58,27153.556,3.5254,3.5254,1.000000,48.46,32.30,-1.0,1632355200
1958384,2021-09-23,688800,78.40,79.44,80.00,81.05,75.85,78.1837,1.60,2.0408,7087.72,55414.441,3.2259,3.2259,1.000000,94.08,62.72,-1.0,1632355200
1958385,2021-09-23,688819,44.89,45.08,44.58,45.20,44.28,44.7130,-0.31,-0.6906,25476.27,113912.164,2.3983,2.3983,1.014728,53.87,35.91,-1.0,1632355200


# 开始进行delay的操作

In [11]:
delay = 30 # 设置delay的天数
one_data['pct_change_shift'] = (one_data['close'].shift(-2) - one_data['close']) / one_data['close'] * 100 # 设置y
one_data = one_data[['timestamp','ticker','open','high','low','close','vwap','volume','pct_chg','turnover','free_turnover','pct_change_shift']]
one_data = one_data.set_index(['timestamp','ticker'])
delay_data = one_data.shift(delay)
array , delay_array = np.array(one_data) , np.array(delay_data)
display(one_data)
display(delay_data)

C:\Users\Xinfei\AppData\Local\Temp/ipykernel_14488/2394363054.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_data['pct_change_shift'] = (one_data['close'].shift(-2) - one_data['close']) / one_data['close'] * 100 # 设置y


,,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,pct_change_shift
timestamp,ticker,,,,,,,,,,
2019-09-23,1,15.34,15.47,15.18,15.38,15.3122,1403282.00,0.2608,0.7428,1.5038,2.405722
2019-09-24,1,15.47,15.54,15.18,15.18,15.2886,1129103.24,-1.3004,0.5977,1.2100,3.491436
2019-09-25,1,15.20,15.85,15.19,15.75,15.5996,1938892.72,3.7549,1.0263,2.0778,0.952381
2019-09-26,1,15.86,16.04,15.66,15.71,15.8491,1686592.58,-0.2540,0.8928,1.8074,-0.763845
2019-09-27,1,15.85,16.00,15.70,15.90,15.8634,1007706.27,1.2094,0.5193,1.0235,1.886792
...,...,...,...,...,...,...,...,...,...,...,...
2021-09-15,1,19.31,20.99,19.19,19.52,19.4779,841618.41,0.7744,0.4337,0.9785,-5.071721
2021-09-16,1,19.30,19.50,18.73,19.08,18.9852,1176953.56,-2.2541,0.6065,1.3684,-5.765199
2021-09-17,1,18.90,19.00,18.41,18.53,18.6572,1124879.63,-2.8826,0.5797,1.3078,-4.587156


,,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,pct_change_shift
timestamp,ticker,,,,,,,,,,
2019-09-23,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-24,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-25,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-26,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-27,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2021-09-15,1,18.04,18.07,17.60,17.81,17.8197,729620.08,-0.4472,0.3760,0.8483,0.280741
2021-09-16,1,17.70,18.05,17.58,17.68,17.7759,621573.88,-0.7299,0.3203,0.7227,7.805430
2021-09-17,1,17.55,17.90,17.45,17.86,17.7260,567454.13,1.0181,0.2924,0.6597,10.470325


In [13]:
delay_array.shape

(487, 10)

In [28]:
day = 3
x , y , x_delay = [] , [], []
# for i in range(0,array.shape[0] - day ,3): # 其中3 代表取数的步长，ex.每两天取一次数，步长为3
for i in range(delay,delay + 3 ,3): # 其中3 代表取数的步长，ex.每两天取一次数，步长为3
    x.append(array[i:i+day,:-1].T)
    y.append(array[i+day-1][-1])
    x_delay.append(delay_array[i:i+day,:-1].T)
x  , y , x_delay= np.array(x) , np.array(y).reshape(-1,1) , np.array(x_delay) # x = (153, 9, 30) , y = (153,1)
x = x.reshape(x.shape[0],1,x.shape[1],x.shape[2]) # x = (153, 1, 9, 30)
x_delay = x_delay.reshape(x_delay.shape[0],1,x_delay.shape[1],x_delay.shape[2]) # x = (153, 1, 9, 30)

x_delay = (x-x_delay)/x_delay
display(x.shape)
display(x_delay)

(1, 1, 9, 3)

array([[[[ 0.0756193 ,  0.05300582,  0.07171053],
         [ 0.06851972,  0.05341055,  0.0384858 ],
         [ 0.06719368,  0.06455863,  0.05727452],
         [ 0.05851756,  0.07575758,  0.0368254 ],
         [ 0.06607803,  0.06521199,  0.0450268 ],
         [-0.32051033, -0.54276422, -0.66211609],
         [-9.52070552, -1.23615811, -1.        ],
         [-0.33844911, -0.55496068, -0.67105135],
         [-0.2628009 , -0.50396694, -0.63341034]]]])